This notebook applies our histogram matching based approach on the survey images and saves outputs in data/surveyimages/model_hist_matching folder.
To see the implementations of the helper functions please check hist_matching_helpers.py file.

Note: Please use runtime type as GPU

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
import numpy as np
import cv2
from os import listdir
from os.path import join as p_join
from os.path import abspath
from pathlib import Path 
import pandas as pd
import matplotlib.pyplot as plt
import glob

PROJDIR = abspath('')
PROJDIR

'/content'

In [3]:
# put the path to the folder here 
# if this is a shared folder: right click on the folder in "Shared with me" directory, and then click "Add shortcut to Drive". 
# then you can access the data from your drive!
PROJDIR = '/content/drive/MyDrive/CP/Computational Photography'
print(PROJDIR)
DATA = p_join(PROJDIR, 'data')

/content/drive/MyDrive/CP/Computational Photography


*Load* Emotion6 dataset

In [4]:
# emotion6 dataset
emotions_e6 = ['anger', 'disgust', "fear", "joy", "sadness", "surprise"]
EM = p_join(DATA, 'Emotion6')
EM = p_join(EM, 'train')
emotion_distr = pd.read_csv(p_join(EM, "ground_truth2.csv"), sep=';')
emotion_distr.columns = ['folder', 'image', 'valence', 'arousal', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']
#change image number to integer
data_types_dict = {'image': np.int64}
# ground truth probability distributions + valence & arousal
emotion_distr = emotion_distr.astype(data_types_dict)
# images
#emotion_prob = [[(cv2.imread(p_join(e, file)), file.split('.')[0]) for file in listdir(e) if file != "test"] for e in emotion_folder]


In [5]:
# Choose reference images as images with strong gt value from emotion6
tmp=emotion_distr.sort_values('anger',ascending=False)
ref_imgs_anger = tmp['image'][0:20].values
ref_imgs_anger_folders = tmp[0:20]['folder'].values
tmp=emotion_distr.sort_values('disgust',ascending=False)
ref_imgs_disgust = tmp['image'][0:20].values
ref_imgs_disgust_folders = tmp[0:20]['folder'].values
tmp=emotion_distr.sort_values('fear',ascending=False)
ref_imgs_fear = tmp['image'][0:20].values
ref_imgs_fear_folders = tmp[0:20]['folder'].values
tmp=emotion_distr.sort_values('joy',ascending=False)
ref_imgs_joy = tmp['image'][0:20].values
ref_imgs_joy_folders = tmp[0:20]['folder'].values
tmp=emotion_distr.sort_values('sadness',ascending=False)
ref_imgs_sadness = tmp['image'][0:20].values
ref_imgs_sadness_folders = tmp[0:20]['folder'].values
tmp=emotion_distr.sort_values('surprise',ascending=False)
ref_imgs_surprise = tmp['image'][0:20].values
ref_imgs_surprise_folders = tmp[0:20]['folder'].values

In [6]:
# Load reference images from emotion6 dataset (Takes some time~2 mins)
emotions_e6 = ['anger', 'disgust', "fear", "joy", "sadness", "surprise"]
EM = p_join(DATA, 'Emotion6')
EM = p_join(EM, 'images')
ref_list_anger = [cv2.imread(e, )for e in EM+"/"+ref_imgs_anger_folders+"/"+[str(int)+'.jpg' for int in ref_imgs_anger]]
ref_list_disgust = [cv2.imread(e, )for e in EM+"/"+ref_imgs_disgust_folders+"/"+[str(int)+'.jpg' for int in ref_imgs_disgust]]
ref_list_fear = [cv2.imread(e, )for e in EM+"/"+ref_imgs_fear_folders+"/"+[str(int)+'.jpg' for int in ref_imgs_fear]]
ref_list_joy = [cv2.imread(e, )for e in EM+"/"+ref_imgs_joy_folders+"/"+[str(int)+'.jpg' for int in ref_imgs_joy]]
ref_list_sadness = [cv2.imread(e, )for e in EM+"/"+ref_imgs_sadness_folders+"/"+[str(int)+'.jpg' for int in ref_imgs_sadness]]
ref_list_surprise = [cv2.imread(e, )for e in EM+"/"+ref_imgs_surprise_folders+"/"+[str(int)+'.jpg' for int in ref_imgs_surprise]]
ref_lists = [ref_list_anger,ref_list_disgust,ref_list_fear,ref_list_joy,ref_list_sadness,ref_list_surprise]#keep all in a list

load model for classification

In [7]:
%ls
%cd /content/drive/MyDrive/CP/Computational Photography
%cd artemis
!pip install -e .

drive/  sample_data/
/content/drive/.shortcut-targets-by-id/1tMW7kRVrirU9R4wxlCq58YDRba-UZ3Tr/Computational Photography
/content/drive/.shortcut-targets-by-id/1tMW7kRVrirU9R4wxlCq58YDRba-UZ3Tr/Computational Photography/artemis
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/.shortcut-targets-by-id/1tMW7kRVrirU9R4wxlCq58YDRba-UZ3Tr/Computational%20Photography/artemis
     |████████████████████████████████| 2.6 MB 14.0 MB/s 
     |████████████████████████████████| 140 kB 68.9 MB/s 
  Running setup.py develop for artemis


In [8]:
from artemis.in_out.neural_net_oriented import torch_load_model, torch_save_model, save_state_dicts
import os.path as osp
save_dir = './model_temp'  # for trained model
checkpoint_file = osp.join(save_dir, 'our_model.pt')

model = torch_load_model(checkpoint_file)    

In [9]:
from google.colab.patches import cv2_imshow
from artemis.neural_models.resnet_encoder import ResnetEncoder
from artemis.neural_models.image_emotion_clf import ImageEmotionClassifier
from artemis.neural_models.mlp import MLP
from PIL import Image
import torch
import random
import warnings
import os.path as osp
import multiprocessing as mp
import torchvision.transforms as transforms

"""
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = torch.nn.KLDivLoss(reduction='batchmean').to(device)
n_emotions=7
img_encoder = ResnetEncoder('resnet34', adapt_image_size=1).unfreeze(level=7, verbose=True)
img_emb_dim = img_encoder.embedding_dimension()

# here we make an MLP closing with LogSoftmax since we want to train this net via KLDivLoss
clf_head = MLP(img_emb_dim, [100, n_emotions], dropout_rate=0.3, b_norm=True, closure=torch.nn.LogSoftmax(dim=-1))

model = ImageEmotionClassifier(img_encoder, clf_head).to(device);
optimizer = torch.optim.Adam([{'params': filter(lambda p: p.requires_grad, model.parameters()), 'lr': 5e-4}])
"""

'\ndevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")\ncriterion = torch.nn.KLDivLoss(reduction=\'batchmean\').to(device)\nn_emotions=7\nimg_encoder = ResnetEncoder(\'resnet34\', adapt_image_size=1).unfreeze(level=7, verbose=True)\nimg_emb_dim = img_encoder.embedding_dimension()\n\n# here we make an MLP closing with LogSoftmax since we want to train this net via KLDivLoss\nclf_head = MLP(img_emb_dim, [100, n_emotions], dropout_rate=0.3, b_norm=True, closure=torch.nn.LogSoftmax(dim=-1))\n\nmodel = ImageEmotionClassifier(img_encoder, clf_head).to(device);\noptimizer = torch.optim.Adam([{\'params\': filter(lambda p: p.requires_grad, model.parameters()), \'lr\': 5e-4}])\n'

Import helper function for histogram matching

In [10]:
%cd /content/drive/MyDrive/CP/Computational Photography
from hist_matching_helpers import transform_image

/content/drive/.shortcut-targets-by-id/1tMW7kRVrirU9R4wxlCq58YDRba-UZ3Tr/Computational Photography


In [11]:
#Load survey images
EM_survey = p_join(DATA, "surveyimages")
survey_images = []
survey_em = []
survey_filenames = []
for filename in listdir(EM_survey):
    img = cv2.imread(p_join(EM_survey,filename))
    if img is not None:
        survey_images.append(img)
        survey_em.append(filename.split("_")[1][0:-4])
        survey_filenames.append(filename)
images = np.array(survey_images)
survey_em = np.array(survey_em)
survey_anger = images[survey_em=='anger']
survey_disgust = images[survey_em=='disgust']
survey_fear = images[survey_em=='fear']
survey_joy = images[survey_em=='joy']
survey_sadness = images[survey_em=='sadness']
survey_surprise = images[survey_em=='surprise']
survey_imgs_all = [survey_anger,survey_disgust,survey_fear,survey_joy,survey_sadness,survey_surprise]
survey_orig_ems = [0, 1, 2, 3 ,4, 5]
survey_target_ems = [3, 3, 3, 4 ,3, 4]
survey_target_names = ["joy", "joy", "joy", "sadness", "joy", "sadness"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if sys.path[0] == '':


In [12]:
#Apply transformation and save images in data/surveyimages/model_hist_matching
idx=0
for i, orig_imgs in enumerate(survey_imgs_all):
    em_orig = survey_orig_ems[i]
    orig_name = emotions_e6[i]
    em_target = survey_target_ems[i]
    target_name = survey_target_names[i]
    for j, img in enumerate(orig_imgs):
      transformed = transform_image(img, ref_lists, em_orig, em_target, model)
      img_name = p_join(p_join(EM_survey, "model_hist_matching"), survey_filenames[idx][:-4]+"_"+target_name+".jpg")
      cv2.imwrite(img_name,transformed)
      idx = idx+1

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
